# Dependency parsing baseline


Dependency parsing is the task of mapping a sentence to a formal representation of its syntactic structure in the form of a dependency tree, which consists of directed arcs between individual words (tokens). Here we will implement a dependency parser baseline based on the arc-standard algorithm and the fixed-window model that we implemented in Lab L3.

## Imports

In [1]:
from batchify import *
from create_vocab import *
from data_handling import *
from parser import *
from projectivize import *
from uas import *
from window_models import *
from taggers import *

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/home/hugo/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
device

device(type='cpu')

## The data set

In [4]:
train_data = Dataset('data/en_gum-ud-train-projectivized.conllu')
dev_data = Dataset('data/en_gum-ud-dev-projectivized.conllu')
test_data = Dataset('data/en_gum-ud-test-projectivized.conllu')

## Train tagger

In [5]:
import torch.optim as optim
import torch
import torch.nn.functional as F

def train_fixed_window(train_data, n_epochs=2, batch_size=100, lr=1e-2):
    vocab_words, vocab_tags = make_vocabs(train_data)
    tagger = FixedWindowTagger(vocab_words, vocab_tags, len(vocab_tags))
    
    optimizer = optim.Adam(tagger.model.parameters(), lr=lr)
    for i in range(n_epochs):
        total_loss = 0
        batch_nr = 0
        for x, y in training_examples_tagger(vocab_words, vocab_tags, train_data, tagger):
            batch_nr += 1
            
            optimizer.zero_grad()
            y_pred = tagger.model.forward(x)
            
            loss = F.cross_entropy(y_pred, y)
            loss.backward()
            total_loss += loss.item()
            optimizer.step()
            if batch_nr % 100 == 1:
                print(total_loss/batch_nr)
                #pass
    return tagger


In [6]:
tagger = train_fixed_window(train_data)

2.945399522781372
1.0833388466646177
0.7836531811389164
0.6509181580868274
0.5735636852179977
0.5267426948288005
0.49444046542172426
0.47488184821673707
0.4538583012760504
0.38657471537590027
0.1895852284967014
0.15158450752682057
0.13304140005087436
0.1299453686372356
0.13090858017624227
0.12813943459771684
0.12680283621798527
0.12723575043986166


In [7]:
accuracy(tagger, dev_data)

0.8806006592601636

## Train parser

In [8]:
import torch.optim as optim
import torch
import torch.nn.functional as F
def train_fixed_parser(train_data, n_epochs=2, batch_size=100, lr=1e-2):
    vocab_words, vocab_tags = make_vocabs(train_data)
    parser = FixedWindowParser(vocab_words, vocab_tags)
    
    optimizer = optim.Adam(parser.model.parameters(), lr=lr)
    for i in range(n_epochs):
        total_loss = 0
        batch_nr = 0
        for x, y in training_examples_parser(vocab_words, vocab_tags, train_data, parser):
            batch_nr += 1
            
            optimizer.zero_grad()
            y_pred = parser.model.forward(x)
            
            loss = F.cross_entropy(y_pred, y)
            loss.backward()
            total_loss += loss.item()
            optimizer.step()
            if batch_nr % 100 == 1:
                print(total_loss/batch_nr)
                
    return parser

In [9]:
parser = train_fixed_parser(train_data)

1.1173102855682373
0.545782717737821
0.46493785031399326
0.42630153879572785
0.4066673758395592
0.39208559009307875
0.369733460978938
0.3574000803937075
0.34172584696878655
0.3368986583625966
0.3307248713238375
0.328935859266408
0.3284109847420062
0.32908533072008894
0.329729473538734
0.3275521129498793
0.3245700177068192
0.3602370023727417
0.2778075862639021
0.25538460921440553
0.24505215875829178
0.23899770467693074
0.23521123437616165
0.2221445962817667
0.21714475368031336
0.2105441237538216
0.20988512667336323
0.2079023242110273
0.20815936254792516
0.20804544997406962
0.20813897515587698
0.20854866270209896
0.20837594026750542
0.2083293306231173


In [10]:
uas(parser, dev_data)

0.6795743043979997

## Testing parser with predicted tags 

In [11]:
def calc_uas_with_tagger_preds(tagger, parser, data):
    correct = 0
    total = 0
    
    new_data = []
    for sent in data:
        pred_tags = tagger.predict(sent)
    
        # Replace gold tags with predicted
        for i , (_, tag) in enumerate(pred_tags):
            sent[i] = (sent[i][0], tag, sent[i][2])
        new_data.append(sent)
        
    return uas(parser, new_data)

In [12]:
calc_uas_with_tagger_preds(tagger, parser, dev_data)

0.6216822669573022